In [1]:
import csv
import numpy as np
import urllib.request
import io

In [2]:
from sklearn.metrics.pairwise import paired_distances
from scipy.sparse import hstack

Читаем csv с векторами предложений

In [4]:
def read_csv_link(response):
    spamreader = csv.reader(response)
    vectors = []
    i = 0
    for row in spamreader:
        if i == 0:
            i += 1
            continue
        row = [float(el) for el in row[1:]]
        vectors.append(row)
    return vectors

Считаем расстояния между векторами предложений

In [5]:
def count_distances_link(file1, file2):
    vectors_1 = read_csv_link(file1)
    vectors_2 = read_csv_link(file2)
    vectors_1 = np.array([np.array(vectors_1[i]) for i in range(len(vectors_1))])
    vectors_2 = np.array([np.array(vectors_2[i]) for i in range(len(vectors_2))])
    dist_1 = vectors_1 - vectors_2
    
    euclidean = paired_distances(vectors_1, vectors_2, metric='euclidean')
    manhattan= paired_distances(vectors_1, vectors_2, metric='manhattan')
    cosine = paired_distances(vectors_1, vectors_2, metric='cosine')

    distances =  np.array([euclidean, manhattan, cosine]).T
    return distances

### Метрики

In [6]:
response_val_1_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_1_1.csv')
response_val_2_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_2_1.csv')
response_val_1_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_1_2.csv')
response_val_2_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_2_2.csv')
response_test_1_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_1_1.csv')
response_test_2_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_2_1.csv')
response_test_1_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_1_2.csv')
response_test_2_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_2_2.csv')

X_val_1 = count_distances_link(io.TextIOWrapper(response_val_1_1), io.TextIOWrapper(response_val_2_1))
X_val_2 = count_distances_link(io.TextIOWrapper(response_val_1_2), io.TextIOWrapper(response_val_2_2))
X_val = np.vstack((X_val_1, X_val_2))

X_test_1 = count_distances_link(io.TextIOWrapper(response_test_1_1), io.TextIOWrapper(response_test_2_1))
X_test_2 = count_distances_link(io.TextIOWrapper(response_test_1_2), io.TextIOWrapper(response_test_2_2))
X_test = np.vstack((X_test_1, X_test_2))

In [8]:
response_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_1_0.csv')
response_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_2_0.csv')
X_train = count_distances_link(io.TextIOWrapper(response_1), io.TextIOWrapper(response_2))
for i in range(1, 69):
    response_1 = urllib.request.urlopen(f'https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_1_{i}.csv')
    response_2 = urllib.request.urlopen(f'https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_2_{i}.csv')
    X = count_distances_link(io.TextIOWrapper(response_1), io.TextIOWrapper(response_2))
    X_train = np.vstack((X_train, X))

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import fbeta_score,make_scorer

In [11]:
train = pd.read_json('snli_1.0_train.jsonl', lines = True)
test = pd.read_json('snli_1.0_test.jsonl', lines = True)
val = pd.read_json('snli_1.0_dev.jsonl', lines = True)
full = {'train': train, 'val':val, 'test': test}

In [13]:
def delete_unsure_labels(data):
    data = data.drop(index = data[data.gold_label == '-'].index)
    return data


for frame in full:
    full[frame] = delete_unsure_labels(full[frame])

In [14]:
def delete_unsure_labels_vec(X, data):
    X = X[data.index]
    return X

In [15]:
train = full['train']
val = full['val']
test = full['test']

In [16]:
X_train = delete_unsure_labels_vec(X_train, train)
X_val = delete_unsure_labels_vec(X_val, val)
X_test = delete_unsure_labels_vec(X_test, test)

In [17]:
y_train = np.array(train['gold_label'])
y_val = np.array(val['gold_label'])
y_test = np.array(test['gold_label'])

In [19]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
clf.fit(X_train, y_train)
y_train_preds = clf.predict(X_train)
f_train = fbeta_score(y_train, y_train_preds, beta=7, average=None,)
y_val_preds = clf.predict(X_val)
f_val = fbeta_score(y_val, y_val_preds, beta=7, average=None,)
print(f'train: {f_train}')
print(f'val: {f_val}')

train: [0.21737106 0.57207067 0.20949664]
val: [0.92043663 0.02754214 0.01701473]


In [18]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
ftwo_scorer = make_scorer(fbeta_score, beta=3, average='weighted')
cross_val_score(clf, X_test, y_test, cv=3, scoring= ftwo_scorer)

c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


array([0.28775785, 0.28881738, 0.28754821])

In [15]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
clf.fit(X_test, y_test)
y_train_preds = clf.predict(X_test)
f_train = fbeta_score(y_test, y_train_preds, beta=7, average=None,)
y_val_preds = clf.predict(X_val)
f_val = fbeta_score(y_val, y_val_preds, beta=7, average=None,)
print(f'train: {f_train}')
print(f'val: {f_val}')

train: [0.09069626 0.78048526 0.16737006]


In [20]:
prfs = precision_recall_fscore_support(y_val, y_val_preds, beta=7, average=None, labels=['contradiction', 'neutral', 'entailment'])
matrix_report = pd.DataFrame(prfs, columns = ['contradiction', 'neutral', 'entailment'], index = ['precision', 'recall', 'fscore', 'support'])
matrix_report['Weighted*'] = 0.5 * matrix_report['contradiction'] +  0.25 * matrix_report['neutral'] +  0.25 * matrix_report['entailment']
matrix_report.T

,precision,recall,fscore,support
contradiction,0.332766,0.954851,0.920437,3278.0
neutral,0.315789,0.016692,0.017015,3235.0
entailment,0.339623,0.027035,0.027542,3329.0
Weighted*,0.330236,0.488357,0.471358,3280.0
